In [9]:
# In this stage, I add all the modules I want to use or import

import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt

from scipy.linalg import orthogonal_procrustes
from scipy.spatial.transform import Rotation as Rscipy
import numpy as np

In [10]:
# STEP 1: MAKING WORLD POINTS and CORNER DETECTION

# first, we create 3D world coordinates of the actual checkerboard based on measurements
# we try to map out each corner in the coordinate system (X, Y, Z) = horizontal, vertical, and depth positions

def make_world_points_3D_custom_origin(inner_cols,
                                       inner_rows,
                                       square_size_m,
                                       cam_distance_m=None,
                                       origin_col=4,
                                       origin_row=2):
    """
    Generate 3D world coordinates for a checkerboard pattern, with a
    custom origin (row,col) corner
    """
    objp = []
    for r in range(inner_rows):
        for c in range(inner_cols):
            # Shift so that (origin_row, origin_col) → (0,0)
            X = (c - origin_col) * square_size_m
            Y = (r - origin_row) * square_size_m

            if cam_distance_m is not None:
                Z = np.sqrt(max(cam_distance_m**2 - X**2 - Y**2, 0))
            else:
                Z = 0.0

            objp.append([X, Y, Z])

    return np.asarray(objp, dtype=np.float64)


# This function detects corners of the checkerboard using the cv2 tool

def detect_corners(image_path, pattern_size):
    """
    Returns (ok, corners Nx2) subpixel-refined if found.
    pattern_size: (cols, rows) of INNER corners, e.g., (7,5).
    """
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    assert img is not None, f"Failed to read {image_path}"
    ok, corners = cv2.findChessboardCorners(img, pattern_size,
                                            flags=cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE)
    if not ok:
        return False, None
    # Subpixel refine
    corners = cv2.cornerSubPix(img, corners, (11,11), (-1,-1),
                               (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 40, 1e-3))
    return True, corners.reshape(-1,2)

In [11]:

# STEP 2: Compute the projection matrix (3D → 2D)
# This is the perspective projection stage where we map 3D world coordinates (X, Y, Z) to image plane coordinates (u, v)
# using the equation: s * [u v 1]^T = P * [X Y Z 1]^T

def normalize_points_2d(pts):
    """Normalize 2D points so centroid is at origin and mean distance is sqrt(2)."""
    pts = np.asarray(pts)
    mean = np.mean(pts, axis=0)
    d = np.mean(np.sqrt(np.sum((pts - mean)**2, axis=1)))
    s = np.sqrt(2) / d
    T = np.array([[s, 0, -s*mean[0]],
                  [0, s, -s*mean[1]],
                  [0, 0, 1]])
    pts_h = np.c_[pts, np.ones(len(pts))]
    pts_n = (T @ pts_h.T).T[:, :2]
    return pts_n, T


def normalize_points_3d(pts):
    """Normalize 3D points so centroid is at origin and mean distance is sqrt(3)."""
    pts = np.asarray(pts)
    mean = np.mean(pts, axis=0)
    d = np.mean(np.sqrt(np.sum((pts - mean)**2, axis=1)))
    s = np.sqrt(3) / d
    T = np.array([[s, 0, 0, -s*mean[0]],
                  [0, s, 0, -s*mean[1]],
                  [0, 0, s, -s*mean[2]],
                  [0, 0, 0, 1]])
    pts_h = np.c_[pts, np.ones(len(pts))]
    pts_n = (T @ pts_h.T).T[:, :3]
    return pts_n, T

# In this stage we calculate a projection matrix to map our world coordinates to (u, v)
# from this projection matrix our instrinsic camera parameters can be extracted
def projection_matrix_dlt(XYZ, uv):
    """
    Funciton to generate P based on XYZ coordinates and uv
    """
    XYZ = np.asarray(XYZ)
    uv = np.asarray(uv)

    # Normalize for stability
    XYZn, T3 = normalize_points_3d(XYZ)
    uvn, T2 = normalize_points_2d(uv)

    # Build linear system A p = 0
    A = []
    for (X, Y, Z), (u, v) in zip(XYZn, uvn):
        Xh = [X, Y, Z, 1]
        A.append([0, 0, 0, 0, *(-u * np.array(Xh)),  v * X, v * Y, v * Z, v])
        A.append([*Xh, 0, 0, 0, 0, -u * X, -u * Y, -u * Z, -u])
    A = np.asarray(A)

    # Solve via SVD
    _, _, VT = np.linalg.svd(A)
    p = VT[-1, :]
    Pn = p.reshape(3, 4)

    # Denormalize
    P = np.linalg.inv(T2) @ Pn @ T3
    P /= P[-1, -1]  # scale normalization
    return P


In [12]:
# STEP 3: now that we have the P matrix, we can extract K 
# K = 3x3 matrix inside P
# P = K [R | t], where K are intrinsic parameters,
# and [R|t] describes camera pose relative to the world

def K_R_t_from_P(P):
    """
    Decompose projection matrix P = K [R|t] using RQ decomposition.

    Parameters
    ----------
    P : (3,4) np.ndarray
        Projection matrix from DLT.

    Returns
    -------
    K : (3,3) np.ndarray
        Camera intrinsic matrix.
    R : (3,3) np.ndarray
        Rotation matrix (world-to-camera).
    t : (3,) np.ndarray
        Translation vector (world-to-camera).
    """
    M = P[:, :3]  # left 3x3 part

    # RQ decomposition via QR on reversed axes

    # scipy's decomp


    R_q, Q_r = np.linalg.qr(np.flipud(M).T)
    Rq = np.flipud(R_q.T)
    Qr = Q_r.T[:, ::-1]
    K = Rq
    R = Qr

    # Ensure positive diagonal entries in K
    T = np.diag(np.sign(np.diag(K)))
    K = K @ T
    R = T @ R

    # Normalize K so that K[2,2] = 1
    K = K / K[2,2]

    # Compute translation
    t = np.linalg.inv(K) @ P[:, 3]

    return K, R, t


# This section generates K (instrinsics) as well as Rotation (R) and translation (t) which are Extrinsics


In [13]:
# STEP 4: Running the pipeline over several images


def calibrate_from_3D(images_glob,
                      inner_cols=9, inner_rows=7,
                      square_size_m=0.03,
                      cam_distance_m=None,
                      origin_col=4, origin_row=2):
    """
    Full 3D DLT-based camera calibration pipeline.
    Uses 3D-2D correspondences to compute projection matrices and intrinsics.

    Parameters
    ----------
    images_glob : str
        Glob pattern for checkerboard calibration images.
    inner_cols, inner_rows : int
        Number of inner corners along width and height.
    square_size_m : float
        Checkerboard square size in meters.
    cam_distance_m : float or None
        Optional camera-to-board distance (used for Z computation).
    origin_col, origin_row : int
        Origin corner indices (0-based, OpenCV convention).

    Returns
    -------
    K_avg : (3,3) np.ndarray
        Averaged intrinsic matrix across all images.
    extrinsics : list[(path, R, t)]
        Rotation and translation per image.
    projections : list[(path, P)]
        Projection matrix per image.
    """

    # 1) Generate 3D world coordinates
    obj_xyz = make_world_points_3D_custom_origin(
        inner_cols, inner_rows, square_size_m,
        cam_distance_m, origin_col, origin_row
    )

    # 2) Loop over images and compute P, K, R, t
    image_paths = sorted(glob.glob(images_glob))
    assert image_paths, f"No images match pattern: {images_glob}"

    Ks, extrinsics, projections = [], [], []

    for path in image_paths:
        ok, img_pts = detect_corners(path, (inner_cols, inner_rows))
        if not ok:
            print(f"[WARN] Corners not found in {path}, skipping.")
            continue

        P = projection_matrix_dlt(obj_xyz, img_pts)
        K, R, t = K_R_t_from_P(P)

        Ks.append(K)
        extrinsics.append((path, R, t))
        projections.append((path, P))
        print(f"[OK] Processed {path}")

    assert Ks, "No successful calibrations."
    K_avg = np.mean(Ks, axis=0)
    K_avg /= K_avg[2,2]  # normalize

    # Average rotation and translation across all images
    R_all = np.array([R for (_, R, _) in extrinsics])
    t_all = np.array([t for (_, _, t) in extrinsics])

    R_mean = np.mean(R_all, axis=0)
    t_mean = np.mean(t_all, axis=0)

    print("\nCalibration Results:")
    print("Average Intrinsic Matrix K:\n", K_avg)
    print("\nAverage Rotation Matrix (approx):\n", R_mean)
    print("Average Translation Vector (approx):\n", t_mean)

    return K_avg, extrinsics, projections

In [14]:
# Run pipeline


# Set Constants
images_glob = "selected_photos/*.jpg"   # path to your checkerboard images
inner_cols = 8
inner_rows = 6 
square_size_m = 30 # 30mm squares for checkerboard
# cam_height_m = 0.3731 # 37.31cm distance between camera and checkerboard
cam_height_m = 373.1
# cam_height_m = None

# Run all
K_avg, extrinsics, projections = calibrate_from_3D(
    images_glob=images_glob,
    inner_cols=inner_cols,
    inner_rows=inner_rows,
    square_size_m=square_size_m,
    cam_distance_m=cam_height_m,
    # cam_distance_m = None,
    origin_col=4,
    origin_row=2
)

print("K:", K_avg)

# --- Rotation matrix orthonormality check ---
print("\nRotation Matrix Orthogonality Check:")
for path, R, t in extrinsics:
    ortho_err = np.linalg.norm(R.T @ R - np.eye(3))
    detR = np.linalg.det(R)
    print(f"{path.split('/')[-1]}: ||RᵀR - I|| = {ortho_err:.6e}, det(R) = {detR:.6f}")


# Try OPEN CV version
# orthonormality check for rotation matrix
# Projection error

[OK] Processed selected_photos/PXL_20251015_204036401.jpg
[OK] Processed selected_photos/PXL_20251015_204038338.jpg
[OK] Processed selected_photos/PXL_20251015_204042061.jpg
[OK] Processed selected_photos/PXL_20251015_204051458.jpg
[OK] Processed selected_photos/PXL_20251015_204055860.jpg
[OK] Processed selected_photos/PXL_20251015_204057569.jpg
[OK] Processed selected_photos/PXL_20251015_204059267.jpg
[OK] Processed selected_photos/PXL_20251015_204101178.jpg
[OK] Processed selected_photos/PXL_20251015_204102905.jpg
[OK] Processed selected_photos/PXL_20251015_204104631.jpg
[OK] Processed selected_photos/PXL_20251015_204108195.jpg
[OK] Processed selected_photos/PXL_20251015_204109931.jpg
[OK] Processed selected_photos/PXL_20251015_204112244.jpg
[OK] Processed selected_photos/PXL_20251015_204114102.jpg
[OK] Processed selected_photos/PXL_20251015_204115675.jpg
[OK] Processed selected_photos/PXL_20251015_204117201.jpg
[OK] Processed selected_photos/PXL_20251015_204118628.jpg
[OK] Processed

In [15]:
# Open CV and custom code comparison

# Recreate world and image points for cv2.calibrateCamera
objp = make_world_points_3D_custom_origin(
    inner_cols, inner_rows, square_size_m,
    cam_distance_m=None, origin_col=4, origin_row=2
)
obj_points = []  # list of Nx3
img_points = []  # list of Nx2

image_paths = sorted(glob.glob(images_glob))
for p in image_paths:
    ok, corners = detect_corners(p, (inner_cols, inner_rows))
    if ok:
        obj_points.append(objp.astype(np.float32))
        img_points.append(corners.astype(np.float32))
    else:
        print(f"[WARN] Skipping {p}")

# Image size must be (width, height)
img = cv2.imread(image_paths[0], cv2.IMREAD_GRAYSCALE)
h, w = img.shape[:2]

print(f"Number of valid calibration images: {len(img_points)}")
assert len(img_points) > 0, "No valid corner detections for OpenCV calibration!"

# Run OpenCV’s built-in calibration
ret, K_cv, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
    obj_points, img_points, (w, h), None, None
)

print("\nOpenCV Intrinsic Matrix (K):\n", K_cv)
print("Distortion Coefficients:\n", dist_coeffs.ravel())
print(f"RMS reprojection error (OpenCV): {ret:.4f}")

# --- Compare our DLT vs OpenCV ---
print("\n=== Comparison Summary ===")
print("Our DLT Intrinsic Matrix (K_avg):\n", K_avg)
print("\nOpenCV Intrinsic Matrix (K_cv):\n", K_cv)

diff_percent = 100 * (K_avg - K_cv) / K_cv
print("\nPercent difference (elementwise):\n", np.round(diff_percent, 2))

Number of valid calibration images: 18

OpenCV Intrinsic Matrix (K):
 [[1.50518378e+04 0.00000000e+00 2.02730379e+03]
 [0.00000000e+00 1.48409101e+04 1.47855129e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Coefficients:
 [ 4.31693790e+00 -5.80920397e+02  3.77347399e-03 -7.98953970e-03
  2.20675387e+04]
RMS reprojection error (OpenCV): 0.8831

=== Comparison Summary ===
Our DLT Intrinsic Matrix (K_avg):
 [[ 9.99944776e-01 -1.23173688e-01 -1.20853264e+01]
 [-2.03441872e-03  1.21267158e+01 -1.23731014e-01]
 [ 1.20861678e+01  1.22599689e-02  1.00000000e+00]]

OpenCV Intrinsic Matrix (K_cv):
 [[1.50518378e+04 0.00000000e+00 2.02730379e+03]
 [0.00000000e+00 1.48409101e+04 1.47855129e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Percent difference (elementwise):
 [[ -99.99    -inf -100.6 ]
 [   -inf  -99.92 -100.01]
 [    inf     inf    0.  ]]


/var/folders/h7/r3_ys_1j43qghpb1wgxjxt3c0000gn/T/ipykernel_27800/163675462.py:41: RuntimeWarning: divide by zero encountered in divide
  diff_percent = 100 * (K_avg - K_cv) / K_cv


In [16]:
# Orthonormality check and reprojection error checks

# orthonormality check
for path, R, t in extrinsics:
    ortho_err = np.linalg.norm(R.T @ R - np.eye(3))
    detR = np.linalg.det(R)
    print(f"{path.split('/')[-1]}: ||RᵀR - I|| = {ortho_err:.6e}, det(R) = {detR:.6f}")

# reprojection error check
print("\n=== Reprojection Error (Custom DLT) ===")

def reprojection_error(P, XYZ, uv_true):
    XYZ_h = np.c_[XYZ, np.ones(len(XYZ))]
    uv_proj_h = (P @ XYZ_h.T).T
    uv_proj = uv_proj_h[:, :2] / uv_proj_h[:, 2, None]
    err = np.linalg.norm(uv_proj - uv_true, axis=1)
    return float(np.mean(err)), float(np.std(err))



# Recreate the same world coordinates used in calibration
obj_xyz = make_world_points_3D_custom_origin(
    inner_cols, inner_rows, square_size_m,
    cam_distance_m=cam_height_m, origin_col=4, origin_row=2
)

for (path, R, t), (_, P) in zip(extrinsics, projections):
    ok, img_pts = detect_corners(path, (inner_cols, inner_rows))
    if not ok:
        continue
    mean_err, std_err = reprojection_error(P, obj_xyz, img_pts)
    print(f"{path.split('/')[-1]}: mean = {mean_err:.2f} px, std = {std_err:.2f} px")

PXL_20251015_204036401.jpg: ||RᵀR - I|| = 2.315623e+05, det(R) = 1228.574296
PXL_20251015_204038338.jpg: ||RᵀR - I|| = 1.160674e+05, det(R) = 488.452208
PXL_20251015_204042061.jpg: ||RᵀR - I|| = 1.666586e+05, det(R) = 706.576362
PXL_20251015_204051458.jpg: ||RᵀR - I|| = 1.804042e+05, det(R) = 800.928592
PXL_20251015_204055860.jpg: ||RᵀR - I|| = 1.428135e+05, det(R) = 635.177027
PXL_20251015_204057569.jpg: ||RᵀR - I|| = 1.705813e+05, det(R) = 757.135659
PXL_20251015_204059267.jpg: ||RᵀR - I|| = 2.096524e+05, det(R) = 1018.930324
PXL_20251015_204101178.jpg: ||RᵀR - I|| = 1.637495e+05, det(R) = 772.693329
PXL_20251015_204102905.jpg: ||RᵀR - I|| = 1.886315e+05, det(R) = 864.697935
PXL_20251015_204104631.jpg: ||RᵀR - I|| = 1.208526e+05, det(R) = 514.582036
PXL_20251015_204108195.jpg: ||RᵀR - I|| = 1.454415e+05, det(R) = 651.726639
PXL_20251015_204109931.jpg: ||RᵀR - I|| = 2.530025e+05, det(R) = 1343.714785
PXL_20251015_204112244.jpg: ||RᵀR - I|| = 2.083751e+05, det(R) = 1033.657860
PXL_2025